In [26]:
# Importar dependencias

from s3fs.core import S3FileSystem
import pandas as pd
import os


In [4]:
# Agregar credenciales
access = "capacitacion"
secret = "capacitacion"
s3 = S3FileSystem(
    key=access,
    secret=secret,
    endpoint_url="http://10.91.160.65:9000"
)

In [8]:
# Exploremos nuestra conexión
buckets = s3.ls('')

# Imprimir los buckets a los que tenemos acceso
for bucket in buckets:
    print(bucket)

capacitacion-lago-datos


In [13]:
# Veamos los archivos dentro de un bucket
files = s3.ls('capacitacion-lago-datos')

# Imprimir archivos dentro del bucket
for file in files:
    print(file)

capacitacion-lago-datos/2024-10-03T11-07_export.csv


Creamos un dataframe de juguete, para escribirlo en minio

In [17]:
# Columnas de juguete
x = [i for i in range(10)]
y = [i + 3 for i in range(10)]

# Escribir archivo con pandas
df = pd.DataFrame({"x": x, "y": y})
df.to_parquet("../data/df_ejemplo.parquet")


In [21]:
# Escribir en minio el archivo parquet
bucket = "capacitacion-lago-datos"
file = "df_ejemplo.parquet"
source_folder = "../data"

minio_path = f"{bucket}/{file}"
local_path = f"{source_folder}/{file}"
s3.put_file(rpath=minio_path,lpath=local_path)

Replicamos varias veces el archivo en nuestro directorio local

In [24]:
# Contiene el archivo creado manualmente
original_file =  pd.read_parquet(local_path)

# Vamos a escribir varias veces el mismo archivo
for i in range(10):
    print(f"Escribiendo archivo {i}")
    original_file.to_parquet(f"../data/replica_{i}.parquet")     

Escribiendo archivo 0
Escribiendo archivo 1
Escribiendo archivo 2
Escribiendo archivo 3
Escribiendo archivo 4
Escribiendo archivo 5
Escribiendo archivo 6
Escribiendo archivo 7
Escribiendo archivo 8
Escribiendo archivo 9


In [30]:
# Escribimos todo en minio
parquet_files =  os.listdir("../data")
bucket = "capacitacion-lago-datos"
for file in parquet_files:
    print(f"--------- Escribiendo en S3 archivo {file}")
    rpath = f"{bucket}/{file}"
    file_path = f"../data/{file}"
    s3.put_file(lpath=file_path,rpath=rpath)




--------- Escribiendo en S3 archivo df_ejemplo.parquet
--------- Escribiendo en S3 archivo replica_0.parquet
--------- Escribiendo en S3 archivo replica_1.parquet
--------- Escribiendo en S3 archivo replica_2.parquet
--------- Escribiendo en S3 archivo replica_3.parquet
--------- Escribiendo en S3 archivo replica_4.parquet
--------- Escribiendo en S3 archivo replica_5.parquet
--------- Escribiendo en S3 archivo replica_6.parquet
--------- Escribiendo en S3 archivo replica_7.parquet
--------- Escribiendo en S3 archivo replica_8.parquet
--------- Escribiendo en S3 archivo replica_9.parquet


In [31]:
# Revisemos que todo haya quedado bien escrito
files = s3.ls('capacitacion-lago-datos')

# Imprimir archivos dentro del bucket
for file in files:
    print(file)


capacitacion-lago-datos/df_ejemplo.parquet
capacitacion-lago-datos/replica_0.parquet
capacitacion-lago-datos/replica_1.parquet
capacitacion-lago-datos/replica_2.parquet
capacitacion-lago-datos/replica_3.parquet
capacitacion-lago-datos/replica_4.parquet
capacitacion-lago-datos/replica_5.parquet
capacitacion-lago-datos/replica_6.parquet
capacitacion-lago-datos/replica_7.parquet
capacitacion-lago-datos/replica_8.parquet
capacitacion-lago-datos/replica_9.parquet
